In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from tqdm import tqdm
tqdm.pandas()
from datetime import datetime

from dateutil.relativedelta import relativedelta
pd.set_option("display.max_columns", None)

# Date format = "%Y-%m-%d" @ datetime type

In [2]:
main2021 = pd.read_csv("Main_Firm_No Loc_Fin_Calc v180225.csv", index_col = 0).drop("FirmID", axis = 1).rename(
    {"FirmAge":"FirmAge", "lnage":"ln_FirmAge"}, axis = 1)

main2021["AsOnDate"] = pd.to_datetime(main2021["AsOnDate"], format = "%Y-%m-%d")

main2021["PercentWomenDir"] = main2021["PercentWomenDir"]*100
main2021["PercentRookieIndep"] = main2021["PercentRookieIndep"]*100
main2021["PercentBusyDir"] = main2021["PercentBusyDir"]*100
main2021["PercentFirstTermIndep"] = main2021["PercentFirstTermIndep"]*100


main2021_sliced = main2021.loc[ main2021["AsOnDate"] >= "2021-03-31" ].drop(
    ["Paid up preference capital_net of forfeited preference capital", "Market Capitalisation",
     "Debt", "Long term borrowings incl current portion",
     "Long term borrowings excl current portion", "Total assets", "BVPS",
     "Shares Outstanding ", "Research development expenses", "P/B "], axis = 1)
    
main2021_sliced

,Symbol,AsOnDate,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,FirstYearIndepPCodeList,IndepPCodeList,FirstYearPCodeList,PCodeList,CountRetiringTermLimit,CountFirstTermIndep,CountDirTurnOver_exits_retire,PercentDirTurnOver_exits_retire,CountDirTurnOver_real_exits,PercentDirTurnOver_real_exits,CountIndepDirTurnOver_exits_retire,PercentIndepDirTurnOver_exits_retire,CountIndepDirTurnOver_real_exits,PercentIndepDirTurnOver_real_exits,CountDirTurnOver_exits_retire_shifted,PercentIndepDirTurnOver_exits_retire_shifted,CountDirTurnOver_real_exits_shifted,PercentDirTurnOver_real_exits_shifted,CountIndepDirTurnOver_exits_retire_shifted,CountIndepDirTurnOver_real_exits_shifted,PercentIndepDirTurnOver_real_exits_shifted,Company Name,Sales,Profit after tax,PBDITA,Return on total assets,Operating profit of non-financial companies,Operating profit of financial companies,Debt to equity ratio,Trade payables_old,Subscribed preference capital,Paid up preference capital_net of forfeited preference capital_old,Paid up preference shares,Total other receivables,Net cash flow from operating activities,Cash outflow due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Cash flow due to purchase of fixed assets,Net cash inflow or outflow from investing activities,Intercorporate loans_old,Deferred tax liability,Net deferred tax liabilities,Total Returns,Market Capitalisation / Enterprise Value,Prowess company code,Entity type,Entity type code,Incorporation year,Industry group,Industry group code,NIC name,NIC code,Ownership group code,Ownership group,Age group,NSE symbol,Head office address,Total Shares_percent_Shares held,Promoters_percent,Indian Promoters_percent,Foreign Promoters_percent,Nonpromoters_percent,NonpromoterInstitutions_percent,Nonpromoter Noninstitutions_percent,BVCommonStock,Office,NoFamilyControl,PBOA_calc,ROA_calc,ln_totalassets,Book value of common stock,TobinQ_Debt1,ln_TobinQ_Debt1,TobinQ_longborrowincl2,TobinQ_longborrowexcl3,ln_TobinQ_longborrowincl2,ln_TobinQ_longborrowexcl3,TobinQ_BVPS,BookLeverage,Debt_TotalAssets,RD to Assets,ln_marcap,FirmAge,ln_FirmAge,govtdummy,findummy,promoterholding20,promoterholding30,promoterholding40
12,20MICRONS,2021-03-31,8,2.079442,1,12.500000,1,62.875000,1,0,7,4,0.500000,4,1,0,3,4,3,1,1,2021,1,1,1,0,1,1,0,2,1,1,14.157534,2.875000,25.000000,0.375000,0.000000,0.500000,0.125000,0.875000,0,1,0,0.000000,[],25.000000,25.000000,[],"['P00027633', 'P00031581', 'P00066605', 'P0009...",[],"['P00027633', 'P00029006', 'P00027661', 'P0002...",0.0,1.0,1.0,0.125,1.0,0.125,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.000,0.0,0.0,0.0,20 Microns Ltd.,4363.2,224.5,561.5,5.46,556.2,208.9,0.46,790.3,NaN,NaN,NaN,8.8,564.6,-95.6,1.8,95.6,-94.7,16.4,235.9,223.6,1.15,0.59,11.0,Public Ltd.,1.020302e+10,1987.0,Minerals,1.010203e+14,Other mining and quarrying,8.0,2.010104e+10,20 Microns Group,Between 1986 and 1990,20MICRONS,"347, G I D C Industrial Estate, Waghodia, Vad...",100.0,44.83,44.83,0.0,55.17,0.00,55.17,1.990864e+09,Vadodara,0,0.134462,0.053761,8.337085,1.990864e+09,0.513235,-0.667021,0.385024,0.350899,-0.954451,-1.047256,-476749.689078,0.277258,0.216217,0.002347,7.123125,34.0,3.526361,0,0,1,1,1
13,20MICRONS,2022-03-31,7,1.945910,1,14.285714,1,60.571429,1,0,6,4,0.571429,3,1,0,3,3,2,1,1,2022,1,1,1,1,1,1,1,2,0,0,12.463014

In [3]:
main2013 = pd.read_csv("Firm level data_locationRavi_21226_final(in).csv").drop(
    "FirmID", axis = 1).rename({"Age":"FirmAge", "LnAge":"ln_FirmAge", "TobinQ1_Debt":"TobinQ_Debt1", "Ln_TobinQ1_Debt":"ln_TobinQ_Debt1",
                                "TobinQ1_borrowings":"TobinQ_longborrowincl2", "Ln_TobinQ1_borrowings":"ln_TobinQ_longborrowincl2",
                                "Indian Promoters_percent":"Indian Promoters_percent", "Foreign Promoters_percent":"Foreign Promoters_percent",
                                "Non-promoters_percent":"Nonpromoters_percent", "Non-promoter Institutions_percent":"NonpromoterInstitutions_percent",
                                "Non-promoter Non-institutions_percent":"Nonpromoter Noninstitutions_percent",
                                "Ln_TotalAssets":"ln_totalassets", "Ln_MarketCap":"ln_marcap"}, axis = 1).drop(
    ["Market Capitalisation", "Debt", "Long term borrowings incl current portion", "Long term borrowings excl current portion", 
     "Total assets"], axis = 1)

				

main2013["TobinQ_longborrowexcl3"] = main2013["TobinQ_longborrowincl2"]
main2013["ln_TobinQ_longborrowexcl3"] = np.log(main2013["TobinQ_longborrowexcl3"])

fincode = [649, 64191, 64192, 64300, 64920, 64990, 65110, 65120, 66190, 66301]
main2013["findummy"] = np.where( main2013["NIC code"].isin(fincode), 1, 0)

govtName = ["Central Govt. - Commercial Enterprises", "Bank of Baroda Group", "Bank of India Group", "Canara Bank Group",
            "G.I.C. Group", "State Govt. - Commercial Enterprises", "State Govt. - Statutory Bodies", "I.D.B.I. Group",
            "Indian Bank Group", "L.I.C. Group", "Punjab National Bank Group", "State Bank of India Group"]
main2013["govtdummy"] = np.where( main2013["Ownership group"].isin(govtName), 1, 0)

main2013["AsOnDate"] = [ datetime.fromordinal(datetime(1900, 1, 1).toordinal() + x - 2) for x in main2013["AsOnDate"]]

main2013

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_6420\746807370.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  main2013 = pd.read_csv("Firm level data_locationRavi_21226_final(in).csv").drop(


,cyId,AsOnDate,Year,Symbol,Company,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,PercentIndepDirTurnOver,FIN_BeginsHere,FIN_Status,cyId4_FIN,Sales,Profit after tax,PBDITA,ln_totalassets,Operating Margin,Return on total assets,ROA_calc,ROS_calc,TobinQ_longborrowincl2,ln_TobinQ_longborrowincl2,TobinQ_Debt1,ln_TobinQ_Debt1,Trade payables,Subscribed preference capital,Paid up preference shares,Total other receivables,ORECTA,Net cash flow from operating activities,CFO,Cash (outflow) due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Investment Expenditure_fixed assets,Net cash inflow or (outflow) from investing activities,Research & development expenses,R&D_percent,Debt to equity ratio,Inter-corporate loans,Inter-corporate loans_total assets,Closing Price,ln_marcap,board size_old,PromoterHolding_old,IDEN_BeginsHere,IDEN_Status,cyId4_IDEN,Prowess company code,Entity type,Entity type code,Incorporation year,FirmAge,ln_FirmAge,Industry group,Industry group code,NIC name,NIC code,NIC two,Industry,Ownership group code,Ownership group,Age group,OWN_BeginsHere,OWN_Status,cyId4_OWN,F_Company_Code,EHDate,Total Shares_percent,Promoters_percent,Indian Promoters_percent,Foreign Promoters_percent,Nonpromoters_percent,NonpromoterInstitutions_percent,Nonpromoter Noninstitutions_percent,Location,TotalDirectors,FirstYearDirectors,PercentageOfFirstYearDirectors,FirstYearNFirmsLocation,First-year director(%)t-1,FirstYearIndependentDirectors,PercentageOfFirstYearIndependentDirectors,FirstYearIndependentNFirmsLocation,First-year Independent director(%)t-1,TobinQ_longborrowexcl3,ln_TobinQ_longborrowexcl3,findummy,govtdummy
0,1,2009-03-31,2009,20MICRONS,20 MICRONS LTD.,11,2.397895,0,0.000000,0,54,4,1,7,0,0.000000,11,0,4,0,7,0,0,0,2009,0,0,0,0,0,0,0,0,0,0,7.18,NaN,0,0,36,63,36,63,0,0,1,0,NaN,0,0,0,~~~,NaN,1.0,1454.8,13.4,144.7,7.098706,11.954726,4.38,1.107072,9.946384,0.177214,0.163151,0.661434,0.507681,153.6,NaN,NaN,1.6,0.001322,1.4,0.001157,-94.1,4.2,-0.074273,-89.9,3.1,0.213088,1.64,NaN,NaN,15.10,5.368310,12.0,50.61,~~~,NaN,1.0,11.0,Public Ltd.,NaN,1987.0,22.0,3.091042,Minerals,1.010000e+14,Other mining and quarrying,8.0,8.0,8.0,NaN,Private (Indian),Between 1986 and 1990,~~~,NaN,1.0,11.0,Mar 31 2009 12:00AM,100.0,47.21,39.93,7.28,52.79,20.8800,31.90,Vadodara,119,18,0.151261,9,0.016807,119,1.0,9,0.111111,0.177214,-1.730396,0,0
1,2,2010-03-31,2010,20MICRONS,20 MICRONS LTD.,10,2.302585,0,0.000000,0,54,2,1,8,0,0.000000,10,0,2,0,8,0,0,0,2010,0,0,0,0,0,0,0,0,0,0,7.80,NaN,0,0,20,80,20,80,0,0,1,0,NaN,0,0,0,~~~,NaN,2.0,1808.6,73.0,238.7,7.367519,15.072299,4.42,4.609459,13.198054,0.353369,0.302597,0.885856,0.634382,158.4,NaN,NaN,41.4,0.026141,32.7,0.020648,-164.5,28.1,-0.086127,-196.5,2.4,0.132699,2.03,NaN,NaN,39.05,6.327276,12.0,54.13,~~~,NaN,2.0,11.0,Public Ltd.,NaN,1987.0,23.0,3.135494,Minerals,1.010000e+14,Other mining and quarrying,8.0,8.0,8.0,NaN,Private (Indian),Between 1986 and 1990,~~~,NaN,2.0,11.0,Mar 31 2010 12:00AM,100.0,51.75,44.53,7.22,48.25,18.9600,29.30,Vadodara,127,27,0.212598,11,0.019327,127,1.0,11,0.090909,0.353369,-1.040243,0,0
2,3,2011-03-31,2011,20MICRONS,20 MICRONS LTD.,10,2.302585,0,0.000000,0,55,1,1,9,0,0.000000,10,0,1,0,9,0,0,0,2011,0,0,0,0,0,0,0,0,0,0,8.80,NaN,0,0,10,90,10,90,0,1,0,0,NaN,0,0,0,~~~,NaN,3.0,2408.3,55.0,256.1,7.

In [4]:
[x for x in main2013.columns.to_list() if x not in main2021_sliced.columns.to_list()]

['cyId',
 'Year',
 'Company',
 'PercentIndepDirTurnOver',
 'FIN_BeginsHere',
 'FIN_Status',
 'cyId4_FIN',
 'Operating Margin',
 'ROS_calc',
 'Trade payables',
 'ORECTA',
 'CFO',
 'Cash (outflow) due to purchase of fixed assets',
 'Investment Expenditure_fixed assets',
 'Net cash inflow or (outflow) from investing activities',
 'Research & development expenses',
 'R&D_percent',
 'Inter-corporate loans',
 'Inter-corporate loans_total assets',
 'Closing Price',
 'board size_old',
 'PromoterHolding_old',
 'IDEN_BeginsHere',
 'IDEN_Status',
 'cyId4_IDEN',
 'NIC two',
 'Industry',
 'OWN_BeginsHere',
 'OWN_Status',
 'cyId4_OWN',
 'F_Company_Code',
 'EHDate',
 'Total Shares_percent',
 'Location',
 'TotalDirectors',
 'FirstYearDirectors',
 'PercentageOfFirstYearDirectors',
 'FirstYearNFirmsLocation',
 'First-year director(%)t-1',
 'FirstYearIndependentDirectors',
 'PercentageOfFirstYearIndependentDirectors',
 'FirstYearIndependentNFirmsLocation',
 'First-year Independent director(%)t-1']

In [5]:
[x for x in main2021_sliced.columns.to_list() if x in main2013.columns.to_list()]

['Symbol',
 'AsOnDate',
 'TotDirCount',
 'LnBoardSize',
 'FemaleDirCount',
 'PercentWomenDir',
 'Women_dummy',
 'AvgAge',
 'CountRookie',
 'Rookie_dummy',
 'CountNonRookie',
 'CountIndep',
 'PercentIndep',
 'CountNonIndep',
 'CountRookieIndep',
 'CountRookieNonIndep',
 'CountNonRookieIndep',
 'CountNonRookieNonIndep',
 'CountPromoterDir',
 'CountBusyDir',
 'Busy_dummy',
 'AsOnYear',
 'IsCeoMD',
 'HasChairman',
 'HasPromoterOnBoard',
 'IsDualityChairmanMD',
 'IsFamilyManaged',
 'IsFamilyChairman',
 'IsFamilyChairmanAndCEO',
 'CountFirstTermDir',
 'CountTurnOverIndepDir',
 'HasIndepTurnOver',
 'AvgTenureInYearsALL',
 'AvgTenureInYearsIndep',
 'PercentRookieIndep',
 'PercentNonRookieIndep',
 'PercentRookieNonIndep',
 'PercentNonRookieNonIndep',
 'PercentRookie',
 'PercentNonRookie',
 'IsRookieBoard',
 'JustOneRookie',
 'TwoOrMoreRookies',
 'PercentRetiringTermLimit',
 'TermLimitRetireesPcodeList',
 'PercentBusyDir',
 'PercentFirstTermIndep',
 'Sales',
 'Profit after tax',
 'PBDITA',
 'Ret

In [6]:
merge = main2013.merge(main2021_sliced, on = [x for x in main2021_sliced.columns.to_list() if x in main2013.columns.to_list()], how = "outer")

C:\Users\SHIVAM\AppData\Local\Temp\ipykernel_6420\1505471315.py:1: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  merge = main2013.merge(main2021_sliced, on = [x for x in main2021_sliced.columns.to_list() if x in main2013.columns.to_list()], how = "outer")


In [7]:
merge

,cyId,AsOnDate,Year,Symbol,Company,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,PercentIndepDirTurnOver,FIN_BeginsHere,FIN_Status,cyId4_FIN,Sales,Profit after tax,PBDITA,ln_totalassets,Operating Margin,Return on total assets,ROA_calc,ROS_calc,TobinQ_longborrowincl2,ln_TobinQ_longborrowincl2,TobinQ_Debt1,ln_TobinQ_Debt1,Trade payables,Subscribed preference capital,Paid up preference shares,Total other receivables,ORECTA,Net cash flow from operating activities,CFO,Cash (outflow) due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Investment Expenditure_fixed assets,Net cash inflow or (outflow) from investing activities,Research & development expenses,R&D_percent,Debt to equity ratio,Inter-corporate loans,Inter-corporate loans_total assets,Closing Price,ln_marcap,board size_old,PromoterHolding_old,IDEN_BeginsHere,IDEN_Status,cyId4_IDEN,Prowess company code,Entity type,Entity type code,Incorporation year,FirmAge,ln_FirmAge,Industry group,Industry group code,NIC name,NIC code,NIC two,Industry,Ownership group code,Ownership group,Age group,OWN_BeginsHere,OWN_Status,cyId4_OWN,F_Company_Code,EHDate,Total Shares_percent,Promoters_percent,Indian Promoters_percent,Foreign Promoters_percent,Nonpromoters_percent,NonpromoterInstitutions_percent,Nonpromoter Noninstitutions_percent,Location,TotalDirectors,FirstYearDirectors,PercentageOfFirstYearDirectors,FirstYearNFirmsLocation,First-year director(%)t-1,FirstYearIndependentDirectors,PercentageOfFirstYearIndependentDirectors,FirstYearIndependentNFirmsLocation,First-year Independent director(%)t-1,TobinQ_longborrowexcl3,ln_TobinQ_longborrowexcl3,findummy,govtdummy,FirstYearIndepPCodeList,IndepPCodeList,FirstYearPCodeList,PCodeList,CountRetiringTermLimit,CountFirstTermIndep,CountDirTurnOver_exits_retire,PercentDirTurnOver_exits_retire,CountDirTurnOver_real_exits,PercentDirTurnOver_real_exits,CountIndepDirTurnOver_exits_retire,PercentIndepDirTurnOver_exits_retire,CountIndepDirTurnOver_real_exits,PercentIndepDirTurnOver_real_exits,CountDirTurnOver_exits_retire_shifted,PercentIndepDirTurnOver_exits_retire_shifted,CountDirTurnOver_real_exits_shifted,PercentDirTurnOver_real_exits_shifted,CountIndepDirTurnOver_exits_retire_shifted,CountIndepDirTurnOver_real_exits_shifted,PercentIndepDirTurnOver_real_exits_shifted,Company Name,Operating profit of non-financial companies,Operating profit of financial companies,Trade payables_old,Paid up preference capital_net of forfeited preference capital_old,Cash outflow due to purchase of fixed assets,Cash flow due to purchase of fixed assets,Net cash inflow or outflow from investing activities,Intercorporate loans_old,Deferred tax liability,Net deferred tax liabilities,Total Returns,Market Capitalisation / Enterprise Value,NSE symbol,Head office address,Total Shares_percent_Shares held,BVCommonStock,Office,NoFamilyControl,PBOA_calc,Book value of common stock,TobinQ_BVPS,BookLeverage,Debt_TotalAssets,RD to Assets,promoterholding20,promoterholding30,promoterholding40
0,1.0,2009-03-31,2009.0,20MICRONS,20 MICRONS LTD.,11,2.397895,0,0.0,0,54.00,4,1,7,0,0.000,11,0,4,0,7,0,0,0,2009,0,0,0,0,0,0,0,0,0,0,7.180000,NaN,0.000000,0.000,36.000,63.000,36.000,63.000,0,0,1,0.000000,NaN,0.000000,0.000000,0.0,~~~,NaN,1.0,1454.8,13.4,144.7,7.098706,11.954726,4.38,1.107072,9.9

In [8]:
calcCol = ["Paid up preference capital_net of forfeited preference capital", "Market Capitalisation",
     "Debt", "Long term borrowings incl current portion",
     "Long term borrowings excl current portion", "Total assets", "BVPS",
     "Shares Outstanding ", "Research development expenses", "P/B "]

In [9]:
merge.set_index(["Symbol", "AsOnDate"], inplace = True)
main2021_calcCol = main2021.set_index(["Symbol", "AsOnDate"]).loc[:, calcCol].copy()
main2021_calcCol

Paid up preference capital_net of forfeited preference capital  \
Symbol    AsOnDate                                                                     
20MICRONS 2009-03-31                                                NaN                
          2010-03-31                                                NaN                
          2011-03-31                                                NaN                
          2012-03-31                                                NaN                
          2013-03-31                                                NaN                
...                                                                 ...                
ZYLOG     2014-03-31                                                NaN                
          2015-03-31                                                NaN                
          2016-03-31                                                NaN                
          2017-03-31                                                NaN                
          2018-03-31                                                NaN                

                      Market Capitalisation    Debt  \
Symbol    AsOnDate                                    
20MICRONS 2009-03-31                 214.50   586.1   
          2010-03-31                 559.63   843.3   
          2011-03-31                 532.40  1048.9   
          2012-03-31                1150.78  1447.7   
          2013-03-31                 991.02  1485.9   
...                                     ...     ...   
ZYLOG     2014-03-31                 634.17  9656.6   
          2015-03-31                 250.72  9873.9   
          2016-03-31                 241.87  9111.0   
          2017-03-31                 259.57     NaN   
          2018-03-31                 156.33     NaN   

                      Long term borrowings incl current portion  \
Symbol    AsOnDate                                                
20MICRONS 2009-03-31                                        NaN   
          2010-03-31                                        NaN   
          2011-03-31                                      438.8   
          2012-03-31                                      694.8   
          2013-03-31                                      831.7   
...                                                         ...   
ZYLOG     2014-03-31                                     3584.6   
          2015-03-31                                     1726.5   
          2016-03-31                                      306.0   
          2017-03-31                                        NaN   
          2018-03-31                                        NaN   

                      Long term borrowings excl current portion  Total assets  \
Symbol    AsOnDate                                                              
20MICRONS 2009-03-31                                        NaN        1210.4   
          2010-03-31                                        NaN        1583.7   
          2011-03-31                                      207.0        1984.2   
          2012-03-31                                      514.2        2548.0   
          2013-03-31                                      647.1        2805.9   
...                                                         ...           ...   
ZYLOG     2014-03-31                                     2692.0       16282.5   
          2015-03-31                                      833.9       14773.4   
          2016-03-31                                      306.0        5875.3   
          2017-03-31                                        NaN           NaN   
          2018-03-31                                        NaN           NaN   

                        BVPS  Shares Outstanding   \
Symbol    AsOnDate                                  
20MICRONS 2009-03-31   25.19           14205248.0   
          2010-03-31   29.03           14331028.0   
          2011-03-31   

In [10]:
merge2 = merge.merge(main2021_calcCol, left_index = True, right_index = True, how = "left").reset_index().drop_duplicates()
merge2

,Symbol,AsOnDate,cyId,Year,Company,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,PercentIndepDirTurnOver,FIN_BeginsHere,FIN_Status,cyId4_FIN,Sales,Profit after tax,PBDITA,ln_totalassets,Operating Margin,Return on total assets,ROA_calc,ROS_calc,TobinQ_longborrowincl2,ln_TobinQ_longborrowincl2,TobinQ_Debt1,ln_TobinQ_Debt1,Trade payables,Subscribed preference capital,Paid up preference shares,Total other receivables,ORECTA,Net cash flow from operating activities,CFO,Cash (outflow) due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Investment Expenditure_fixed assets,Net cash inflow or (outflow) from investing activities,Research & development expenses,R&D_percent,Debt to equity ratio,Inter-corporate loans,Inter-corporate loans_total assets,Closing Price,ln_marcap,board size_old,PromoterHolding_old,IDEN_BeginsHere,IDEN_Status,cyId4_IDEN,Prowess company code,Entity type,Entity type code,Incorporation year,FirmAge,ln_FirmAge,Industry group,Industry group code,NIC name,NIC code,NIC two,Industry,Ownership group code,Ownership group,Age group,OWN_BeginsHere,OWN_Status,cyId4_OWN,F_Company_Code,EHDate,Total Shares_percent,Promoters_percent,Indian Promoters_percent,Foreign Promoters_percent,Nonpromoters_percent,NonpromoterInstitutions_percent,Nonpromoter Noninstitutions_percent,Location,TotalDirectors,FirstYearDirectors,PercentageOfFirstYearDirectors,FirstYearNFirmsLocation,First-year director(%)t-1,FirstYearIndependentDirectors,PercentageOfFirstYearIndependentDirectors,FirstYearIndependentNFirmsLocation,First-year Independent director(%)t-1,TobinQ_longborrowexcl3,ln_TobinQ_longborrowexcl3,findummy,govtdummy,FirstYearIndepPCodeList,IndepPCodeList,FirstYearPCodeList,PCodeList,CountRetiringTermLimit,CountFirstTermIndep,CountDirTurnOver_exits_retire,PercentDirTurnOver_exits_retire,CountDirTurnOver_real_exits,PercentDirTurnOver_real_exits,CountIndepDirTurnOver_exits_retire,PercentIndepDirTurnOver_exits_retire,CountIndepDirTurnOver_real_exits,PercentIndepDirTurnOver_real_exits,CountDirTurnOver_exits_retire_shifted,PercentIndepDirTurnOver_exits_retire_shifted,CountDirTurnOver_real_exits_shifted,PercentDirTurnOver_real_exits_shifted,CountIndepDirTurnOver_exits_retire_shifted,CountIndepDirTurnOver_real_exits_shifted,PercentIndepDirTurnOver_real_exits_shifted,Company Name,Operating profit of non-financial companies,Operating profit of financial companies,Trade payables_old,Paid up preference capital_net of forfeited preference capital_old,Cash outflow due to purchase of fixed assets,Cash flow due to purchase of fixed assets,Net cash inflow or outflow from investing activities,Intercorporate loans_old,Deferred tax liability,Net deferred tax liabilities,Total Returns,Market Capitalisation / Enterprise Value,NSE symbol,Head office address,Total Shares_percent_Shares held,BVCommonStock,Office,NoFamilyControl,PBOA_calc,Book value of common stock,TobinQ_BVPS,BookLeverage,Debt_TotalAssets,RD to Assets,promoterholding20,promoterholding30,promoterholding40,Paid up preference capital_net of forfeited preference capital,Market Capitalisation,Debt,Long term borrowings incl current portion,Long term borrowings excl current portion,Total assets,BVPS,Shares Outstanding,Research development expenses,P/B
0,20MICRONS,2009-03-31,1.0,2009.0,20 MICRONS LTD.,

# FIRM ID

In [11]:
firmID = merge2["Symbol"].drop_duplicates().to_frame().reset_index(drop = True).reset_index().rename({"index":"FirmID"}, axis = 1)
firmID

,FirmID,Symbol
0,0,20MICRONS
1,1,21STCENMGM
2,2,3IINFOTECH
3,3,3MINDIA
4,4,3PLAND
...,...,...
2883,2883,ZFCVINDIA
2884,2884,ZIMLAB
2885,2885,ZOMATO
2886,2886,ZUARIIND


In [12]:
merge3 = firmID.merge(merge2, on = "Symbol", how = "inner")
merge3

,FirmID,Symbol,AsOnDate,cyId,Year,Company,TotDirCount,LnBoardSize,FemaleDirCount,PercentWomenDir,Women_dummy,AvgAge,CountRookie,Rookie_dummy,CountNonRookie,CountIndep,PercentIndep,CountNonIndep,CountRookieIndep,CountRookieNonIndep,CountNonRookieIndep,CountNonRookieNonIndep,CountPromoterDir,CountBusyDir,Busy_dummy,AsOnYear,IsCeoMD,HasChairman,HasPromoterOnBoard,IsDualityChairmanMD,IsFamilyManaged,IsFamilyChairman,IsFamilyChairmanAndCEO,CountFirstTermDir,CountTurnOverIndepDir,HasIndepTurnOver,AvgTenureInYearsALL,AvgTenureInYearsIndep,PercentRookieIndep,PercentNonRookieIndep,PercentRookieNonIndep,PercentNonRookieNonIndep,PercentRookie,PercentNonRookie,IsRookieBoard,JustOneRookie,TwoOrMoreRookies,PercentRetiringTermLimit,TermLimitRetireesPcodeList,PercentBusyDir,PercentFirstTermIndep,PercentIndepDirTurnOver,FIN_BeginsHere,FIN_Status,cyId4_FIN,Sales,Profit after tax,PBDITA,ln_totalassets,Operating Margin,Return on total assets,ROA_calc,ROS_calc,TobinQ_longborrowincl2,ln_TobinQ_longborrowincl2,TobinQ_Debt1,ln_TobinQ_Debt1,Trade payables,Subscribed preference capital,Paid up preference shares,Total other receivables,ORECTA,Net cash flow from operating activities,CFO,Cash (outflow) due to purchase of fixed assets,Cash inflow due to sale of fixed assets,Investment Expenditure_fixed assets,Net cash inflow or (outflow) from investing activities,Research & development expenses,R&D_percent,Debt to equity ratio,Inter-corporate loans,Inter-corporate loans_total assets,Closing Price,ln_marcap,board size_old,PromoterHolding_old,IDEN_BeginsHere,IDEN_Status,cyId4_IDEN,Prowess company code,Entity type,Entity type code,Incorporation year,FirmAge,ln_FirmAge,Industry group,Industry group code,NIC name,NIC code,NIC two,Industry,Ownership group code,Ownership group,Age group,OWN_BeginsHere,OWN_Status,cyId4_OWN,F_Company_Code,EHDate,Total Shares_percent,Promoters_percent,Indian Promoters_percent,Foreign Promoters_percent,Nonpromoters_percent,NonpromoterInstitutions_percent,Nonpromoter Noninstitutions_percent,Location,TotalDirectors,FirstYearDirectors,PercentageOfFirstYearDirectors,FirstYearNFirmsLocation,First-year director(%)t-1,FirstYearIndependentDirectors,PercentageOfFirstYearIndependentDirectors,FirstYearIndependentNFirmsLocation,First-year Independent director(%)t-1,TobinQ_longborrowexcl3,ln_TobinQ_longborrowexcl3,findummy,govtdummy,FirstYearIndepPCodeList,IndepPCodeList,FirstYearPCodeList,PCodeList,CountRetiringTermLimit,CountFirstTermIndep,CountDirTurnOver_exits_retire,PercentDirTurnOver_exits_retire,CountDirTurnOver_real_exits,PercentDirTurnOver_real_exits,CountIndepDirTurnOver_exits_retire,PercentIndepDirTurnOver_exits_retire,CountIndepDirTurnOver_real_exits,PercentIndepDirTurnOver_real_exits,CountDirTurnOver_exits_retire_shifted,PercentIndepDirTurnOver_exits_retire_shifted,CountDirTurnOver_real_exits_shifted,PercentDirTurnOver_real_exits_shifted,CountIndepDirTurnOver_exits_retire_shifted,CountIndepDirTurnOver_real_exits_shifted,PercentIndepDirTurnOver_real_exits_shifted,Company Name,Operating profit of non-financial companies,Operating profit of financial companies,Trade payables_old,Paid up preference capital_net of forfeited preference capital_old,Cash outflow due to purchase of fixed assets,Cash flow due to purchase of fixed assets,Net cash inflow or outflow from investing activities,Intercorporate loans_old,Deferred tax liability,Net deferred tax liabilities,Total Returns,Market Capitalisation / Enterprise Value,NSE symbol,Head office address,Total Shares_percent_Shares held,BVCommonStock,Office,NoFamilyControl,PBOA_calc,Book value of common stock,TobinQ_BVPS,BookLeverage,Debt_TotalAssets,RD to Assets,promoterholding20,promoterholding30,promoterholding40,Paid up preference capital_net of forfeited preference capital,Market Capitalisation,Debt,Long term borrowings incl current portion,Long term borrowings excl current portion,Total assets,BVPS,Shares Outstanding,Research development expenses,P/B
0,0,20MICRONS,2009-03-31,1.0,2009.0,20 MICR

In [13]:
merge3.to_csv("Merged_FirmLevel_Old_New v200225.csv")